In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import sys
from datetime import datetime, date
import os
import re
from IPython.display import clear_output

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:96.0) Gecko/20100101 Firefox/96.0'}
FQDN = 'https://m*******.**/'
responce = requests.get(FQDN)

print("request denied") if responce.status_code != 200 else print("ok")

In [ ]:
def get_html(url, params=None):
    r = requests.get(url, headers=header, params=params)
    return r

In [ ]:
def flatten(L: list) -> list:
    '''рекурентное преобразование списка списков списков... в плоский одноранговый список'''
    if len(L) > sys.getrecursionlimit():
        sys.setrecursionlimit(len(L)*10)
    if L == []:
        return L
    if isinstance(L[0], list):
        return flatten(L[0]) + flatten(L[1:])
    rez = list(L[:1] + flatten(L[1:]))
    return rez

In [ ]:
def str_to_date(str_date: str) -> datetime.date:
    '''принимает год/день/месяц ('22/20/04') , возвращает в формате datetime'''
    temp_date = [d for d in str_date.split('/')]
    temp_date.reverse()
    temp_date = '-'.join(temp_date)
    return date.fromisoformat(temp_date)

In [ ]:
def urls_on_page(page_url: str) -> list:
    '''возвращает список дат и ссылок на статьи со страницы'''
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    short_urls = [e.a.get('href') for e in soup.find_all('h4', class_='news-item-title')]
    fqdn_urls = ['/'.join(base_url.split('/')[:3]) + elem for elem in short_urls]
    
    return fqdn_urls

In [ ]:
def flatten_column(df: pd.core.frame.DataFrame, column: str) -> pd.core.frame.DataFrame:
    '''извлекает значения в ячейках датафрейма из списочного вида в текстовый'''

    for idx in range(df.shape[0]):

        if type(df.loc[idx,column]) is list:
            df.loc[idx,column] = ', '.join(df.loc[idx,column]) if len(df.loc[idx,column]) > 0 else ''

    return df

In [ ]:
def read_conf(fqdn: str, start_id: int = 0, start_date: str = '2022-01-01'):
    file_name = 'aggregator.ini'
    domain = FQDN.split('/')[2]
    start_date = date.fromisoformat(start_date)
    if not os.path.exists(file_name):
        open(file_name, 'x')
    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if domain in row[0]:
                try:
                    start_date = date.fromisoformat(row[1])
                except (ValueError):
                    start_date = start_date
                try:
                    start_id = abs(int(float(row[2])))
                except (ValueError):
                    start_id = start_id
                    
    return start_date

In [ ]:
def write_conf(fqdn: str, end_id: int = 0, end_date: str = str(datetime.now().date())):
    domain = fqdn.split('/')[2]
    
    file_name = 'aggregator.ini'
    fin = open(file_name, "rt")
    data = fin.read()
    domain_list = []
    for elem in data.split('\n'):
        domain_list.append(elem.split(',')[0])
        if elem.split(',')[0] == domain:
            data = data.replace(domain + ',' + elem.split(',')[1] +',' + elem.split(',')[2], 
                                domain + ',' + end_date + ',' + str(end_id))
            break
    
    new_row = domain + ',' + end_date + ',' + str(end_id)
    if domain not in domain_list:
        data += new_row if len(domain_list[0]) == 0 else '\n' + new_row

    fin.close()
    fin = open(file_name, "wt")
    fin.write(data)
    fin.close()

In [ ]:
month_list = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
           'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']

In [ ]:
start_date = read_conf(FQDN) #, start_date = '2022-02-23')
start_date

---

###### Раздел "Новости"

In [ ]:
base_url = 'https://m*******.**/press-center/news/?SECTION_ID=&arrFilter_ff%5BNAME%5D=&arrFilter_DATE_ACTIVE_FROM_1=&arrFilter_DATE_ACTIVE_FROM_2=' \
+ datetime.now().date().strftime("%d-%m-%Y").replace('-','.') + '&set_filter=поиск&PAGEN_1='
start_page = base_url + str(1)

In [ ]:
dates=[]
urls=[]
uop = urls_on_page(start_page) # все новостные ссылки с первой разводящей страницы
urls.extend(uop)
for page_url in uop:
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    for e in soup.find_all('div', class_='post-date'):
        day = e.text.strip().split(',')[0].split(' ')[0]
        month = str(month_list.index(e.text.strip().split(',')[0].split(' ')[1]) + 1)
        if len(month) < 2:
            month = '0' + month
        if len(day) < 2:
            day = '0' + day
        year = e.text.strip().split(',')[0].split(' ')[2].split('\n')[0]
        dates.append(day + '/' + month + '/' + year)
        break

In [ ]:
count=1
while str_to_date(dates[-1]) > start_date:
    count += 1
    next_page = base_url + str(count)
    uop = urls_on_page(next_page)
    urls.extend(uop)
    for page_url in uop:
        response = get_html(page_url)
        soup = BeautifulSoup(response.text, 'lxml')
        for e in soup.find_all('div', class_='post-date'):
            day = e.text.strip().split(',')[0].split(' ')[0]
            month = str(month_list.index(e.text.strip().split(',')[0].split(' ')[1]) + 1)
            if len(month) < 2:
                month = '0' + month
            if len(day) < 2:
                day = '0' + day
            year = e.text.strip().split(',')[0].split(' ')[2].split('\n')[0]
            clear_output(wait=True)  
            print(day + '/' + month + '/' + year)
            dates.append(day + '/' + month + '/' + year)
            break
# Количество новых статей с даты последнего запуска агрегатора
news_sum = sum([True for date_news in dates if str_to_date(date_news) > start_date])

print(f'С даты последнего запуска агрегатора [{start_date}] опубликовано {news_sum} статей.')

In [ ]:
titles=[]
for page_url in urls[:news_sum]:
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    titles.append([e for e in soup.find('h1')][0])

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox()

In [ ]:
content = []
ext_urls = []
uop = []

for url in urls[:news_sum]:
    driver.get(url)
    for item in driver.find_elements(By.CLASS_NAME, 'post-body'):
        post_body = item.text
    for footer in driver.find_elements(By.CLASS_NAME, 'post-footer'):
        footer = footer.text
    for element in driver.find_elements(By.CLASS_NAME, "post-body > p > a"):
        uop.append(element.get_attribute('href'))
    
    ext_urls.append(uop)
    content.append(re.sub(r'\s+', ' ', post_body.replace(footer,'').replace('\n',' ')))
    uop = []
    clear_output(wait=True)
    print(url)
    print(len(content), len(ext_urls))

In [ ]:
driver.close()

In [ ]:
df1 = pd.DataFrame()
df1['Дата'] = dates[:news_sum]
df1['Раздел'] = 'Новости'
df1['Заголовок'] = titles
df1['Текст'] = content
df1['Внешние ссылки'] = ext_urls
df1['URL статьи'] = urls[:news_sum]

In [ ]:
# корректируем столбцы 'Текст' и 'Внешние ссылки'
for column in list(df1)[3:5]:
    df1 = flatten_column(df1,column)
df1.head(3)

In [ ]:
file_name = base_url.split('/')[2].split('.')[0] +'.'+ base_url.split('/')[2].split('.')[1] + '_' + str(datetime.now().date()) + '.xlsx'
with pd.ExcelWriter(file_name) as writer:
    df1.to_excel(writer, sheet_name='Новости', index=False)

In [ ]:
#обновление даты в файле aggregator.ini
write_conf(FQDN)

---